In [2]:
import re

# short example for our precentage logic

exm_1_string = "The GC-content is 50%, and the pH is 7"
matches = re.findall(r"\%", exm_1_string)
if(len(matches) == 1):
    print("String contains exactly one %")


String contains exactly one %


In [3]:
from Bio import Entrez

Entrez.email = "bene6@hotmail.de"  # Always tell NCBI who you are

abstracts_with_one_perc = []

# Search PubMed for articles with "GC-content" in the title
for i in range(0, 200, 20):
    handle = Entrez.esearch(db="pubmed", term="GC content bacterial[Title]", retstart=i, retmax=20)
    record = Entrez.read(handle)
    ids = record["IdList"]
    # Process the IDs

    # Get the list of article IDs
    article_ids = record["IdList"]

    # Fetch the abstracts for the articles
    handle = Entrez.efetch(db="pubmed", id=article_ids, rettype="abstract")
    records = Entrez.read(handle)

    # Print the abstracts
    noAbstract = 0
    for record in records["PubmedArticle"]:
        try:
            temp_record = record["MedlineCitation"]["Article"]
            matches = re.findall(r"\%", temp_record["Abstract"]["AbstractText"][0])
            if(len(matches) == 1):
                abstracts_with_one_perc.append(temp_record)
            pass
        except KeyError:
            print("No abstract available")
            noAbstract += 1
    print("No abstract available for " + str(noAbstract) + " articles "  + "by total of " + str(len(article_ids)) + " articles")


No abstract available
No abstract available for 1 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available for 0 articles by total of 20 articles
No abstract available
No abstract available for 1 articles by total of 20 articles


In [4]:
records["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0]

'In this study, Illumina Miseq sequencing of 16S rRNA gene amplicon was performed on sediments collected from Krossfjorden, Arctic for analyzing the bacterial community structure. Metagenome contained 15,936 sequences with 5,809,491 bp size and 53% G+C content. Metagenome sequence information are now available at NCBI under the Sequence Read Archive (SRA) database with accession no. SRP159159. Taxonomic hits distribution from MG-RAST analysis revealed the dominance of Alpha- and Gamma-subdivisions of Proteobacteria (88.89%) along with Bacteriodetes (8.89%) and Firmicutes (2.22%). Predominant species were <i>Alteromonadales bacterium</i> TW-7 (24%), <i>Pseudoalteromonas haloplanktis</i> (20%) and <i>Pseudoalteromonas</i> spp. SM9913 (18%). MG-RAST assisted analysis also detected the presence of a variety of marine taxa like <i>Bacteriodes, Pseudovibrio, Marinobacter, Idiomarina, Teredinibacter,</i> etc. which take part in key ecological functions and biogeochemical activities of Arctic 

In [5]:
records["PubmedArticle"][0]["MedlineCitation"]["Article"].keys()
records["PubmedArticle"][0]["MedlineCitation"]["Article"]["PublicationTypeList"]
# records["PubmedArticle"][0]["MedlineCitation"].keys()
# records["PubmedArticle"][0]["MedlineCitation"].keys()
# records["PubmedArticle"][0].keys()


# records["PubmedArticle"][0]["PubmedData"].keys()

[StringElement('Journal Article', attributes={'UI': 'D016428'})]

In [6]:
records["PubmedArticle"][0]["MedlineCitation"]["Article"]["ELocationID"]

[StringElement('10.1016/j.dib.2018.11.101', attributes={'EIdType': 'doi', 'ValidYN': 'Y'})]

In [7]:
# abstracts_with_one_perc[0]
matches = re.findall(r"\%", abstracts_with_one_perc[0]["Abstract"]["AbstractText"][0])
if(len(matches) > 1):
    print("String contains more than one percent symbol")

In [8]:
len(abstracts_with_one_perc)

33

In [9]:
abstracts_with_one_perc[0]

DictElement({'ELocationID': [StringElement('10.1016/j.margen.2023.101018', attributes={'EIdType': 'doi', 'ValidYN': 'Y'}), StringElement('S1874-7787(23)00010-7', attributes={'EIdType': 'pii', 'ValidYN': 'Y'})], 'ArticleDate': [DictElement({'Year': '2023', 'Month': '02', 'Day': '24'}, attributes={'DateType': 'Electronic'})], 'Language': ['eng'], 'Journal': {'ISSN': StringElement('1876-7478', attributes={'IssnType': 'Electronic'}), 'JournalIssue': DictElement({'Volume': '68', 'PubDate': {'Year': '2023', 'Month': 'Apr'}}, attributes={'CitedMedium': 'Internet'}), 'Title': 'Marine genomics', 'ISOAbbreviation': 'Mar Genomics'}, 'ArticleTitle': 'Complete genome analysis of Bacillus subtilis TY-1 reveals its biocontrol potential against tobacco bacterial wilt.', 'Pagination': {'StartPage': '101018', 'MedlinePgn': '101018'}, 'Abstract': {'AbstractText': ['Bacillus subtilis TY-1 was isolated from 2000\xa0m-deep sea sediments of the Western Pacific Ocean, which was found to exhibit strong antagon

In [11]:
hopefull_dicts = []

# give the one <i> tag which is the nearest to the % sign

for pre_x in abstracts_with_one_perc:
    x = pre_x["Abstract"]["AbstractText"][0]
    
    # Extract the content between the tags
    result = re.search(r"<i>(.*?)</i>", x)
    if result:
        hopefully_bacterial = result.group(1)
    else:
        hopefully_bacterial = None


    # Extract the precentage value
    result = re.search(r"([\d\.]+\%)", x)
    if result:
        hopefully_percentage_for_bacterial = result.group(1)
    else:
        hopefully_percentage_for_bacterial = None

    is_valuable_data = ""
    while(is_valuable_data != "y" and is_valuable_data != "n" and is_valuable_data != "m"):
        is_valuable_data = input("Is the data for " + str(hopefully_bacterial) + " with " + str(hopefully_percentage_for_bacterial) + " correct? (y/n/m)")

    hopefull_dict = {hopefully_bacterial: hopefully_percentage_for_bacterial, "is_valuable_data": is_valuable_data}

    hopefull_dicts.append(hopefull_dict)

# show only dicts with a value and a key
# hopefull_dicts = [x for x in hopefull_dicts if list(x.keys())[0] is not None and list(x.values())[0] is not None]
hopefull_dicts
# how do we verify?





KeyboardInterrupt: Interrupted by user